In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("phase3_aadhaar_early_warning_final.csv")
df['early_warning'] = df['early_warning'].astype(bool)
df['predicted_load_status'] = df['predicted_load_status'].astype(str)

df.head(20)


,pincode,forecast_day,predicted_load_score,predicted_load_status,anomaly_risk,early_warning
0,100000,1,0.040736,Normal,0.084261,False
1,100000,2,0.040736,Normal,0.084261,False
2,100000,3,0.040736,Normal,0.084261,False
3,100000,4,0.040736,Normal,0.084261,False
4,100000,5,0.040736,Normal,0.084261,False
5,100000,6,0.040736,Normal,0.084261,False
6,100000,7,0.040736,Normal,0.084261,False
7,100000,8,0.040736,Normal,0.084261,False
8,100000,9,0.040736,Normal,0.084261,False
9,100000,10,0.040736,Normal,0.084261,False


In [5]:
def recommend_action(row):
    if row['early_warning'] and row['predicted_load_status'] == 'Critical Overload':
        return 'Deploy mobile Aadhaar van'
    
    elif row['early_warning'] and row['predicted_load_status'] == 'Emerging Stress':
        return 'Extend centre working hours'
    
    elif row['anomaly_risk'] > 0.7:
        return 'Inspect centre & biometric devices'
    
    else:
        return 'Maintain normal operations'


In [8]:
df['recommended_action'] = df.apply(recommend_action, axis=1)

df[['pincode', 'forecast_day', 'recommended_action']].head(15)


,pincode,forecast_day,recommended_action
0,100000,1,Maintain normal operations
1,100000,2,Maintain normal operations
2,100000,3,Maintain normal operations
3,100000,4,Maintain normal operations
4,100000,5,Maintain normal operations
5,100000,6,Maintain normal operations
6,100000,7,Maintain normal operations
7,100000,8,Maintain normal operations
8,100000,9,Maintain normal operations
9,100000,10,Maintain normal operations


In [9]:
priority_map = {
    'Deploy mobile Aadhaar van': 'High',
    'Extend centre working hours': 'Medium',
    'Inspect centre & biometric devices': 'Medium',
    'Maintain normal operations': 'Low'
}

df['action_priority'] = df['recommended_action'].map(priority_map)


In [11]:
pin_actions = (
    df.groupby(['pincode', 'recommended_action'])
      .size()
      .reset_index(name='days_required')
)

pin_actions.head(10)


,pincode,recommended_action,days_required
0,100000,Maintain normal operations,14
1,110001,Maintain normal operations,14
2,110002,Deploy mobile Aadhaar van,1
3,110002,Extend centre working hours,2
4,110002,Maintain normal operations,11
5,110003,Maintain normal operations,14
6,110004,Maintain normal operations,14
7,110005,Extend centre working hours,3
8,110005,Maintain normal operations,11
9,110006,Deploy mobile Aadhaar van,2


In [14]:
action_summary = (
    df['recommended_action']
      .value_counts()
      .reset_index()
      .rename(columns={'index': 'action', 'recommended_action': 'count'})
)

action_summary


,count,count
0,Maintain normal operations,225422
1,Extend centre working hours,34389
2,Deploy mobile Aadhaar van,13637


In [15]:
def simulate_intervention(load_score, action):
    if action == 'Deploy mobile Aadhaar van':
        return load_score * 0.60   # 40% load reduction
    elif action == 'Extend centre working hours':
        return load_score * 0.75   # 25% reduction
    elif action == 'Inspect centre & biometric devices':
        return load_score * 0.85   # 15% reduction
    else:
        return load_score          # no change


In [16]:
df['post_intervention_load'] = df.apply(
    lambda x: simulate_intervention(
        x['predicted_load_score'],
        x['recommended_action']
    ),
    axis=1
)


In [18]:
df['load_reduction'] = (
    df['predicted_load_score'] - df['post_intervention_load']
)

df[['predicted_load_score', 'post_intervention_load', 'load_reduction']].head(15)


,predicted_load_score,post_intervention_load,load_reduction
0,0.040736,0.040736,0.0
1,0.040736,0.040736,0.0
2,0.040736,0.040736,0.0
3,0.040736,0.040736,0.0
4,0.040736,0.040736,0.0
5,0.040736,0.040736,0.0
6,0.040736,0.040736,0.0
7,0.040736,0.040736,0.0
8,0.040736,0.040736,0.0
9,0.040736,0.040736,0.0


In [20]:
phase4_final_df = df[
    [
        'pincode',
        'forecast_day',
        'predicted_load_score',
        'predicted_load_status',
        'anomaly_risk',
        'early_warning',
        'recommended_action',
        'action_priority',
        'post_intervention_load',
        'load_reduction'
    ]
]

phase4_final_df.head(30)


,pincode,forecast_day,predicted_load_score,predicted_load_status,anomaly_risk,early_warning,recommended_action,action_priority,post_intervention_load,load_reduction
0,100000,1,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
1,100000,2,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
2,100000,3,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
3,100000,4,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
4,100000,5,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
5,100000,6,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
6,100000,7,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
7,100000,8,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
8,100000,9,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000
9,100000,10,0.040736,Normal,0.084261,False,Maintain normal operations,Low,0.040736,0.000000


In [21]:
phase4_final_df.to_csv(
    "phase4_aadhaar_optimization_recommendations.csv",
    index=False
)

print("✅ Phase 4 optimization CSV exported successfully")


✅ Phase 4 optimization CSV exported successfully


In [2]:
import pandas as pd

df = pd.read_csv(
    "C:/Users/rohit/OneDrive/Desktop/uidai/finals datasets/phase4_aadhaar_optimization_recommendations.xls"
)

unique_pins = df['pincode'].nunique()
print(f"Total unique PIN codes in Phase 4 dataset: {unique_pins}")


Total unique PIN codes in Phase 4 dataset: 19532
